# Import Libraries

In [1]:
!git clone https://github.com/venkyyuvy/image_classifier.git

fatal: destination path 'image_classifier' already exists and is not an empty directory.


In [2]:
cd image_classifier/S7

/content/image_classifier/S7


In [3]:
! nvidia-smi

/bin/bash: nvidia-smi: command not found


In [4]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Data Transformations

We first start with defining our data transformations. We need to think what our data is and how can we augment it to correct represent images which it might not see otherwise.


In [5]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.RandomRotation((-7.0, 7.0), fill=([1, 1, 1,])),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) 
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


# Dataset and Creating Train/Test Split

In [6]:
train_data = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

batch_size=256
kwargs = {
    'batch_size': batch_size,
    'shuffle': True,
    'num_workers': 2,
    'pin_memory': True}

train_loader = torch.utils.data.DataLoader(train_data, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


# Dataloader Arguments & Test/Train Dataloaders


# The model
Let's start with the model we first saw

In [24]:
def get_layer(layer_type, in_channel=None, out_channel=None,
              dropout_value=0.05, norm='batch', group=1):
    norm_map = {'batch': nn.BatchNorm2d,
     'layer': nn.GroupNorm,
     'group': nn.GroupNorm} #norm_layer(4, out_channel),
    norm_layer = norm_map[norm]
    if layer_type == 'C':
        return nn.Sequential(
            nn.Conv2d(in_channels=in_channel,
                      out_channels=out_channel,
                      kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            norm_layer(out_channel, out_channel),
            nn.Dropout(dropout_value)
        )
    elif layer_type == 'c':
        return nn.Conv2d(in_channels=in_channel,
                        out_channels=out_channel,
                        kernel_size=(1, 1), padding=0, bias=False)
    elif layer_type == 'P':
        return nn.MaxPool2d(2, 2)
    elif layer_type == 'G':
        return nn.AdaptiveAvgPool2d(output_size=1)

In [25]:
class Net(nn.Module):
    def __init__(self, schema, channels, norm='batch', dropout_value=0.01):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()
        for layer_type, channel_in, channel_out in zip(
            schema, [3, *channels], channels):
            self.layers.append(get_layer(
                layer_type, channel_in, channel_out,
                dropout_value, norm))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

# Model Params
Can't emphasize on how important viewing Model Summary is.
Unfortunately, there is no in-built model visualizer, so we have to take external help

In [26]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net(
    list('CCcPCCCcPCCCGc'),
    [64, 32, 16, 16,
     8, 16, 16, 32, 32,
     8, 16, 32, 32, 10],
    norm='layer').to(device)
summary(model, input_size=(3, 32, 32))


cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
              ReLU-2           [-1, 64, 32, 32]               0
         GroupNorm-3           [-1, 64, 32, 32]             128
           Dropout-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]          18,432
              ReLU-6           [-1, 32, 32, 32]               0
         GroupNorm-7           [-1, 32, 32, 32]              64
           Dropout-8           [-1, 32, 32, 32]               0
            Conv2d-9           [-1, 16, 32, 32]             512
        MaxPool2d-10           [-1, 16, 16, 16]               0
           Conv2d-11            [-1, 8, 16, 16]           1,152
             ReLU-12            [-1, 8, 16, 16]               0
        GroupNorm-13            [-1, 8, 16, 16]              16
          Dropout-14            [-1

# Training and Testing

Looking at logs can be boring, so we'll introduce **tqdm** progressbar to get cooler logs.

Let's write train and test functions

# Let's Train and test our model

In [ ]:
from torch.optim.lr_scheduler import StepLR
from model import train, test

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=4, gamma=0.05)

EPOCHS =  20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

EPOCH: 0


Loss=2.3022890090942383 Batch_id=91 Accuracy=9.94:  46%|████▋     | 91/196 [01:40<01:48,  1.03s/it]

In [ ]:
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=4, gamma=0.05)

EPOCHS =  20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=1.5161426067352295 Batch_id=195 Accuracy=35.63: 100%|██████████| 196/196 [00:21<00:00,  8.94it/s]



Test set: Average loss: 1.4583, Accuracy: 4569/10000 (45.69%)

EPOCH: 1


Loss=1.1219896078109741 Batch_id=195 Accuracy=51.06: 100%|██████████| 196/196 [00:21<00:00,  9.14it/s]



Test set: Average loss: 1.2310, Accuracy: 5460/10000 (54.60%)

EPOCH: 2


...


Test set: Average loss: 0.8067, Accuracy: 7098/10000 (70.98%)

EPOCH: 18


Loss=0.5310043692588806 Batch_id=195 Accuracy=73.86: 100%|██████████| 196/196 [00:20<00:00,  9.57it/s]



Test set: Average loss: 0.9199, Accuracy: 6820/10000 (68.20%)

EPOCH: 19


Loss=0.7671285271644592 Batch_id=195 Accuracy=74.35: 100%|██████████| 196/196 [00:20<00:00,  9.54it/s]



Test set: Average loss: 0.8015, Accuracy: 7153/10000 (71.53%)

